Importation des librairies

In [2]:
import numpy as np
import pandas as pd
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [1]:
root_path = '/Volumes/Samsung_T5/JULIETTE/data-CORD'


# Recherche du meilleur article

Ouverture du fichier contenant les metadatas.

In [ ]:
metadata_path = f'{root_path}/metadata.csv'
metadata_processed_path=f'{root_path}/metadata_processed.csv'

In [ ]:
meta_df = pd.read_csv(metadata_path)
#meta_df = pd.read_csv(metadata_processed_path)
meta_df.head(5)

In [ ]:
meta_df.info()

Il faut relier les articles au fichier des metadatas. Cela est possible grâce aux colonnes ("pdf_json_files" et "pmc_json_files"). La fonction ci-dessous permet d'ouvrir un article en donnant le chemin d'accès (que l'on récupère dans les métadatas).

In [ ]:
def open_article(path):
    file=open(root_path+"/"+path,'r')
    article=json.load(file)
    file.close()
    return article

In [ ]:
def make_text(article):
    text_list = []
    for entry in article['body_text']:
        text_list.append(entry['text'])
        text_list.append("\n")
    
    text_full=''.join(text_list)
    return text_full

In [ ]:
path=meta_df["pdf_json_files"][0]
article=open_article(path)

## Critères de sélection 

On va distinguer deux catégories d'articles, ceux ayant un texte complet et ceux n'en n'ayant pas.

Pour noter un article on va prendre en compte :
- sa date de parution
- la présence des mots-clés dans le titre, l'abstract et le texte complet avec des poids différents. Cette notation doit dépendre de la taille du texte voir du regroupement
- le nombre de citations (pas présent dans les métadonnées mais à récupérer).

### Date de parution

In [ ]:
def date(x):
    try:
        return int(x[:4])
    except TypeError:
        return np.NaN

In [ ]:
meta_df["date"]=meta_df["publish_time"].apply(lambda x: date(x))

In [ ]:
meta_df["date"].describe()

In [ ]:
ax=meta_df["date"].value_counts().plot(kind='bar',figsize=(12,10))
ax.set_title("date de publication")
ax.set_xlabel("année")
ax.set_ylabel("nombre de publications")

On peut remarquer une explosion des publications en 2020 (ce qui est normal, c'est un corpus rassemblant des articles sur le COVID).

La note sur le critère de parution doit être d'autant plus faible que l'article à été publié il y a longtemps. Ainsi on peut appliquer la notation :
$$note=\frac{1}{2020-AnnéePublication+1}$$
Ainsi un article publié en 2020 aura 1, en 2019 aura 1/2 et en 2011 aura 0.1
Remarque: ce critère est peut être un peu trop sélectif et à tendance à défavoriser énormément les anciens article.

In [ ]:
def critère_date(date):
    try:
        return 1/(2020-date+1)
    except ZeroDivisionError:
        return 0

In [ ]:
meta_df["note_date"]=meta_df["date"].apply(lambda x:critère_date(x))

### Critère sur les mots-clés

Pour savoir si un article traite potentiellement d'un sujet on prend en compte l'abstract, le titre et le texte complet (quand il y en a un). Les poids de la présence des mots-clés dans ceux-ci ne doivent pas être les mêmes.

On utilise la fréquence pour noter la présence d'un mot clé (dans le titre, l'abstract ou le texte complet). Naturellement comme en longueur titre>abstract>texte alors les fréquences seront largement plus fortes dans le titre.

Pour effectuer la recherche, il faut utiliser une "lemmatisation" et une élimination des tirets/majuscules pour éviter de rater certain mots car leur forme change. On applique le processus sur la recherche et les textes.

#### Preprocessing

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [ ]:
# à faire si besoin
#nltk.download('wordnet') 

In [ ]:
lemmatizer=WordNetLemmatizer()
stop=stopwords.words('english')

def preprocessing(x): #tokenisation,stop-words,lemmatization ...
    clean = re.sub(r'['+string.punctuation + '’—”'+']', "", x.lower())
    clean_text= re.sub(r'\W+', ' ', clean)
    a=""
    for i in clean_text.split():
        if i not in stop :
            a+=str(lemmatizer.lemmatize(i))+' '
    return a

On créé de nouvelles colonnes à partir du titre et de l'abstract

In [ ]:
meta_df["title_process"]=meta_df["title"].apply(lambda x: preprocessing(str(x)))

In [ ]:
meta_df["abstract_process"]=meta_df["abstract"].apply(lambda x: preprocessing(str(x)))

In [ ]:
#meta_df.to_csv(root_path+"/metadata_processed.csv") à décommenter lors de la première ouverture, permet de ne pas avoir à refaire toutes les transformations plus tard.

On créé le compteur qui donne la fréquence des mots-clés dans l'abstract (ou le texte).

In [ ]:
def counter(keywords,text): #keywords est une phrase
    keys=preprocessing(keywords).split()
    text=str(text)
    if text=="NaN": #élimine 
        return -1
    a=text.split()
    if len(a)==0:
        return 0
    c=0
    for i in a:
        if i in keys:
            c+=1
    
    return c/len(a)


In [ ]:
def counter_title(keywords,text): #keywords est une phrase
    keys=preprocessing(keywords).split()
    text=str(text)
    if text=="NaN":
        return -1
    a=text.split()
    if len(a)==0:
        return 0
    c=0
    for i in keys:
        if i in a:
            c+=1
    
    return c/len(keys)

In [ ]:
meta_df_short=meta_df[0:1000].copy() #on se limite aux 1000 premiers pour les tests.

In [ ]:
keywords="factor risk smoking"

In [ ]:
meta_df_short["freq_title"]=meta_df_short["title_process"].apply(lambda x:counter_title(keywords,x))
meta_df_short["freq_abstract"]=meta_df_short["abstract_process"].apply(lambda x:counter(keywords,x))
meta_df_short["freq_added"]=meta_df_short[["freq_title","freq_abstract"]].apply(lambda x: x["freq_title"]+x["freq_abstract"],axis=1)

On additionne maintenant les notes et on trie par ordre décroissant

In [ ]:
meta_df_short["note"]=meta_df_short[["freq_added","note_date"]].apply(lambda x: x["freq_added"]+x["note_date"],axis=1)

In [ ]:
meta_df_sorted=meta_df_short.sort_values(by=["note"],ascending=False)
meta_df_sorted[["title","date","freq_added","note"]].head()


#### Nombre de citations
On prend en compte le nombre d'articles sur lesquels s'appuie notre article à noter. Plus celui-ci citera des articles, meilleure sera sa note. Pour la note on peut imaginer se baser sur l'article ayant le plus de citations pour ensuite noter les autres par rapport à lui.

In [ ]:
def has_full_text(x):
    if x["pdf_json_files"]==True and x["pmc_json_files"]==True:
        return False
    return True

In [ ]:
meta_bool=meta_df_short[["pdf_json_files","pmc_json_files"]].isnull()
meta_bool["has_full"]=meta_bool.apply(lambda x:has_full_text(x),axis=1)

In [ ]:
meta_df_full_text=meta_df_short.loc[meta_bool["has_full"]]

In [ ]:
meta_df_full_text.head()

Il y a 58 456 articles qui ont des textes complets. On a déjà une fonction qui à partir d'un chemin d'accès ouvre un article, on crée une fonction qui fait appel à celle-ci.

Une fois que l'article est ouvert on accède au nombre de référence par "ref_entries" et "bib_entries". Faut-il prendre en compte le nombre de citations ou de référence ?
- référence : sur quoi se base le travail
- citation : allusion à un travail déjà accompli 

Il semble qu'il faille plutôt prendre en compte plutot le nombre de citations.

In [ ]:
def get_path(path):
    a=""
    for i in range (len(path)):
        if path[i]==";":
            break
        a+=path[i]
    return a

In [ ]:
set_ref=set()
dico_ref=dict()

In [ ]:
def add_ref(article):
    ref=article["bib_entries"]

    for i in ref.keys():
        title=preprocessing(ref[i]["title"])
        if title in set_ref:
            dico_ref[title]+=1
        else:
            dico_ref[title]=1
            set_ref.add(title)

In [ ]:
def count_ref(x):
    try:
        path=x["pdf_json_files"]
        article=open_article(get_path(str(path)))
       
    except FileNotFoundError:
        path=x["pmc_json_files"]
        article=open_article(get_path(str(path)))
    add_ref(article)
    return "done"

In [ ]:
dico_ref=dict()
set_ref=set()
meta_df_full_text["ref"]=meta_df_full_text[["pdf_json_files","pmc_json_files"]].apply(lambda x: count_ref(x),axis=1)

In [ ]:
def link_ref(x):
    title=x
    if title in set_ref:
        return dico_ref[title]
    return 0
    

In [ ]:
meta_df_full_text["nb_ref_linked"]=meta_df_full_text["title_process"].apply(lambda x: link_ref(x))


In [ ]:
meta_df_full_text.sort_values(by=["nb_ref_linked"],ascending=False)

In [ ]:
"""path=meta_df["pdf_json_files"][0]
article=open_article(path)
add_ref(article)
print(dico_ref)"""

In [ ]:
liste=[]
for i in dico_ref.keys():
    liste.append(dico_ref[i])
print(max(liste))
print(len(dico_ref))
len(set_ref)

In [ ]:
meta_df_full_text["nb_ref"]=meta_df_full_text[["pdf_json_files","pmc_json_files"]].apply(lambda x: get_ref(x),axis=1)

In [ ]:
meta_df_full_text["nb_ref"].describe()

In [ ]:
meta_df_full_text.to_csv(root_path+"/metadata_processed_full_text.csv") #enregistrement du tableau avec les nouvelles informations

On fait une notation linéaire entre celui ayant le plus de référence (il a une note de 1 alors que celui en ayant moins à une note de 0).

In [ ]:
max_ref=meta_df_full_text["nb_ref"].max()
print(max_ref)

In [ ]:
meta_df_full_text["note_ref"]=meta_df_full_text["nb_ref"].apply(lambda x:x/max_ref)

In [ ]:
meta_df_full_text["note_ref"].describe()

Cette notation ne semble pas très cohérente (il semble avoir une publication avec un très très grands nombre de référence ce qui fausse le tout)

In [ ]:
meta_df_full_text.sort_values(by=["nb_ref"],ascending=False)

### Construction du dictionnaire des références

In [ ]:
ref=article["bib_entries"]
print(ref)
#for i in ref.keys():
#    print(ref[i]["title"]+"\n")

# Transformation du titre et abstract

In [ ]:
def remove_stopwords(x):
    a=""
    for i in x.split():
        if i not in stop :
            a+=i+' '
    return a

In [ ]:
def clean_text(article):
    clean1 = re.sub(r'['+string.punctuation + '’—”'+']', "", article.lower())
    return re.sub(r'\W+', ' ', clean1)

In [ ]:

def lem(x):
    a=""
    for i in x.split():
        a+=str(lemmatizer.lemmatize(i))+" "
    return a
        
    

On va maintenant transformer les abstracts et les titres pour qu'ils ne contiennent plus de ponctuation, plus de tirets ni de mots de liaison. On peut utiliser de plus un tokennizer.

In [ ]:
meta_df3=meta_df2.drop(columns=['sha', 'source_x','doi', 'pmcid', 'pubmed_id','license', 'publish_time', 'authors', 'journal', 'mag_id','who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files','url', 's2_id'])

In [ ]:
meta_df3=meta_df3.dropna()

In [ ]:
meta_df3["abstract_token"]=meta_df3["abstract"].apply(lambda x :clean_text(x))

In [ ]:
meta_df3["title_token"]=meta_df3["title"].apply(lambda x :clean_text(x))

In [ ]:
meta_df3.drop(columns=["title","abstract"])

In [ ]:
meta_df3.info()

Maintenant il faut créer un dictionnaire qui contient l'ensemble des mots puis faire un décompte des mots pour chaque article afin d'obtenir un tableau de nombre plus facilement interpretable.
Il faut aussi enlever les stopwords qui ne permettent pas de différencier les articles.

In [ ]:
stop=stopwords.words('english')

In [ ]:
meta_df3["title_stop"]=meta_df3["title_token"].apply(lambda x:remove_stopwords(x))
meta_df3["abstract_stop"]=meta_df3["abstract_token"].apply(lambda x:remove_stopwords(x))


Les mots les plus fréquents ne sont pas surprenants (Covid, patient...) et n'apprennent pas grand chose. 

Est-ce qu'on pourrait supprimer les mots (covid, sarscov2, coronavirus) car tous les articles parlent a priori du sujet) ?


On a des articles "serieux" donc on suppose qu'il n'y a pas de fautes d'orthographes (du moins que la quantité est négligeable).

"freq" est un tableau qui pour chaque mot à son nombre d'apparition dans l'ensemble du corpus.
Il faudrait la même chose (à deux dimensions) pour chaque article.

In [ ]:
meta_df3["title_lem"]=meta_df3["title_stop"].apply(lambda x:lem(x))
meta_df3["abstract_lem"]=meta_df3["abstract_stop"].apply(lambda x:lem(x))


In [ ]:
freq=pd.Series(''.join(meta_df3["abstract_lem"]).split()).value_counts()

In [ ]:
freq.head(50)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(meta_df3["title"])

Text_counts est une matrice, chaque ligne représente un titre et chaque colonne un mot. Ainsi le coefficient ij de le matrice est le nombre de fois que le mot n°j est contenu dans le titre n°i.

Comment à partir de la matrice connaitre de quel mot on parle ? Je pense que cela n'est pas utile (c'est juste une visualisation abstraite à exploiter par l'algorithme).

In [ ]:
meta_df3.info()

In [ ]:
text_counts[0,1]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

In [ ]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(meta_df3["title"])

kmeans = KMeans(n_clusters=10).fit(tfidf)